In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!apt-get install sox libsndfile1 ffmpeg
!pip install wget
!pip install text-unidecode

# ## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
!mkdir configs

# NeMoモデルにおけるアダプタサポート
NeMoでは、モデルを訓練した後、特定のタスク向けにファインチューニングすることが一般的です。これはモデルのパラメータ数が数百万程度の場合には合理的なアプローチです。しかし、数億から数十億パラメータ規模のモデルを扱う場合、このアプローチは急速に非現実的になります。
このようなシナリオに対する潜在的な解決策として、大規模なモデルのファインチューニングが非現実的になった場合、私たちは特定のドメインやタスクに特化させるために[Adapter](https://arxiv.org/abs/1902.00751)を活用します。Adapterは元のモデルのパラメータ総数のほんの一部しか必要とせず、ファインチューニングにおいてはるかに効率的です。
このチュートリアルでは、torch.nn.Moduleを更新してAdapterをサポートする方法と、さらにその応用として、NeMoモデルのコンポーネントに対してAdapterサポートを有効にする方法について解説します。

## アダプターとは何か？
アダプターはシンプルな概念です。下図のように1つのアダプターを図示できます。最も単純な形態では、入力次元（$D$）を小さなボトルネック次元（$H$）に圧縮する残差フィードフォワード層であり、$R^D \text{->} R^H$の計算を行い、活性化関数（例えばReLU）を適用した後、別のフィードフォワード層で$R^H \text{->} R^D$のマッピングを行います。この出力は、単純な残差接続を介して入力に加算されます。
<div align="center"><img src="https://mermaid.ink/img/pako:eNptkLFqwzAQhl9F3ORAPDSjA4EUx6RgXEjbycpwWOdG1JaMfEoakrx7ZcfpUKrlxH_fz4d0gcoqggTqxp6qAzoW76k0Ipx1-WI6z3sRxyuRF1GOZ3KisK6d3YG8GFdZ9hRJeLbMDRmqvkRGpDLrTuiUiEWUigBtlyIVqzBnEqZ66I39dcX6iKytKXeUf-wn-286QoFeBMvmu0PTD-EfyXaQpP9JFmP_1XN4S3kfD8W4ue6o18pjc52gYQlzaMm1qFX4msuQSOADtSQhCdfaOupZgjS3QPpOIdNGabYOkhqbnuaAnu3b2VSQsPP0gFKNnw7bibr9AJkZdXU" height=100% /></div>
-----
このようなアダプタモジュールは通常、初期化時にアダプタの初期出力が常にゼロとなるように設定されます。これにより、このようなモジュールを追加したことによって元のモデルの性能が低下するのを防ぎます。

## 標準アーキテクチャのエミュレーション
このチュートリアルでは、既存のアーキテクチャにアダプターサポートを追加する方法をデモンストレーションします。
ここでは、単純な多層パーセプトロンを用いて実装した基本的なモデルに焦点を当てます。ただし、このモデル自体は標準的なエンコーダ・デコーダ構造をエミュレートします（音声認識、自然言語処理、機械翻訳など、複数の分野で一般的に使用されているアーキテクチャです）。
また、データセット、データローダー、損失関数、評価指標、およびPyTorch Lightningの「ステップ」（トレーナー、検証、テスト）の実装は省略します。

In [ ]:
import torch
import torch.nn as nn
from nemo.core import NeuralModule, ModelPT

from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, dim: int = 50):
        super().__init__()

        self.fc = torch.nn.Linear(dim, dim)
        self.ln = torch.nn.LayerNorm(dim)

    def forward(self, x):
        x = self.fc(x)
        x = self.ln(x)
        return x

class ResidualMLP(torch.nn.Module):
  def __init__(self, dim: int, num_layers: int):
    super().__init__()

    self.dim = dim
    self.num_layers = num_layers
    self.layers = nn.ModuleList([MLP(dim) for _ in range(num_layers)])
  
  def forward(self, x):
    input = x
    for layer in self.layers:
      x = layer(x)
      x = x + input
      input = x
    return x

-----次に、2つの「モジュール」を持つシンプルなモデルを実装します：

In [ ]:
class SimpleModel(ModelPT):
    def __init__(self, cfg, trainer=None):
        super().__init__(cfg, trainer=trainer)

        self.encoder = instantiate(cfg.encoder)  # type: ResidualMLP
        self.decoder = instantiate(cfg.decoder)  # type: ResidualMLP
        self.projection = nn.Linear(self.decoder.dim, cfg.out_features)

    def forward(self, x):
        y = self.encoder(x)
        z = self.decoder(y)
        out = self.projection(z)
        return out

    def list_available_models(cls):
        return []

    def setup_training_data(self, train_data_config):
        pass

    def setup_validation_data(self, val_data_config):
        pass

## 基本モデルの初期化
上記のモデルは、エンコーダとデコーダブロックの2つのコンポーネントから構成されるシンプルな残差MLPネットワークです。実際のタスクでは十分な性能を発揮できないかもしれませんが、このデモンストレーションには十分です。
次に、このモデル用の設定を生成するヘルパーを作成し、生成した設定を使用して新しいモデルを作成してみましょう！

In [ ]:
def get_classpath(cls):
    return f'{cls.__module__}.{cls.__name__}'

def get_model_config(dim=512):
    config = OmegaConf.create(
        {
            'in_features': dim,
            'out_features': 10,
            'encoder': {'_target_': get_classpath(ResidualMLP), 'dim': dim, 'num_layers': 4},
            'decoder': {'_target_': get_classpath(ResidualMLP), 'dim': dim, 'num_layers': 2},
        }
    )
    return config

In [ ]:
dim = 512
model_cfg = get_model_config(dim)
model = SimpleModel(model_cfg)
model.summarize()

In [ ]:
# Check if the forward pass works !
with torch.no_grad():
  input_data = torch.randn(8, dim)
  out = model(input_data)
  print(out.shape)

# アダプターの組み込み - モジュールごとに
基本的なモデルが完成し、順方向パスが正常に実行できるようになった今、モデルとそのモジュールにアダプターサポートを追加できます。レイヤーごとに段階的に実装していきましょう。
アダプターサポートの追加を検討する際、私たちは逆方向から作業します。最も低いレベルで使用されるモジュールから開始し、アダプターのメソッドを最上位の Model から最下位レベルのモジュール/レイヤーへと順に転送するチェーンを構築します。

# 最下位レベルモジュールにおけるアダプタサポート
モデルチェーンを逆方向に辿ると、`Linear`層と`LayerNorm`層を生成する`MLP`モジュールに到達します。ここでは、`nemo.core.adapter_mixins`内で利用可能な`AdapterModuleMixin`をこのMLPモジュールに追加します。
一般的にはモジュールのコードを直接更新することが推奨されますが、他にも実装方法があります（チュートリアルの後半で詳しく説明します）。
-----

## `mixin`とは何ですか？`mixin`とは一般的に、**他のクラスに継承される**クラス、あるいは**他のクラスに追加機能を提供する**クラスを指す用語です_。ただし単独では使用できません_。mixinは、複数の継承を通じてクラスに追加機能を組み込むための比較的安全な方法と緩やかに解釈できます。

In [ ]:
from nemo.core import adapter_mixins

In [ ]:
help(adapter_mixins.AdapterModuleMixin)

In [ ]:
# NOTE: See the *two* classes being inherited here !
class MLP(torch.nn.Module, adapter_mixins.AdapterModuleMixin):
    def __init__(self, dim: int = 50):
        super().__init__()

        self.fc = torch.nn.Linear(dim, dim)
        self.ln = torch.nn.LayerNorm(dim)

    def forward(self, x):
        x = self.fc(x)
        x = self.ln(x)

        # The only necessary change to the module code !
        if self.is_adapter_available():
          x = self.forward_enabled_adapters(x)
        return x

    # add a utility method to calculate number of parameters (or we could simple extend nemo.core.NeuralModule instead)
    @property
    def num_weights(self):
      num: int = 0
      for p in self.parameters():
          if p.requires_grad:
              num += p.numel()
      return num

-----
これで完了です！ほぼすべてのアダプターに対応したMLPレイヤーが完成しました！以下ではいくつかのアダプター機能を実際に試して、このチュートリアルをさらに進めていく際に期待できる機能の一端をお見せします。

## モジュールレベルアダプターの実験
ここでは、先ほど拡張した `MLP` モデルをインスタンス化し、`AdapterModuleMixin` クラスを通じて追加されたすべての機能を探求します。追加の補助コードをほとんど記述することなく！

-----
まず、`MLP`モジュールを作成し、アダプタを追加する前の訓練可能なパラメータ数を表示しましょう

In [ ]:
mlp = MLP(dim)

print(mlp)
print("Num trainable parameters (without adapters):", mlp.num_weights)

## Adapter Modules
次に、このモジュールにアダプタを1つまたは2つインポートして追加してみましょう！まずNeMoの`common`コレクションから`adapter_modules`をインポートします。このモジュールには、他のtorch.nn.Moduleにアタッチ可能な事前定義済みのAdapterモジュールが含まれています！

In [ ]:
from nemo.collections.common.parts import adapter_modules

In [ ]:
# Next we look at one of the adapter modules - the LinearAdapter
linear_adapter = adapter_modules.LinearAdapter(in_features=dim, dim=5)
print(linear_adapter)

-----通常はこのモジュールを直接使用するのではなく、Config Dataclassを`AdapterModuleMixin`のメソッドに渡すことになります。以下に使用例を示します -

## [任意] アダプターコンポーネントの構築
線形アダプターモジュールは、作成可能なアダプターの唯一のタイプではありません！PyTorchでは、任意のtorch.nn.Moduleをアダプターコンポーネントに変換できます。
例えば、既存のPyTorchモジュールをアダプターコンポーネントに変換することが可能です。以下のセクションは**任意**ですが、独自のアダプターを作成したい場合には推奨されます。

------まず、簡単なPyTorchモジュールから始めましょう。

In [ ]:
class SimpleModule(torch.nn.Module):
  def __init__(self, size: int):
    super().__init__()
    self.size = size
    self.model = torch.nn.Sequential(
        torch.nn.Linear(size, size, bias=False),
        torch.nn.Identity(),
    )
  
  def forward(self, x):
    return self.model(x)

### Adapter Strategy
アダプターモジュールは、本質的にはPyTorchのモジュールそのものです。PyTorchのモジュールと同様に、入力テンソルを受け取り、何らかの演算を実行した後、結果を返します。
アダプターを統合する方法は複数あります：残差として追加する、要素ごとに乗算する、入力と連結する（末尾または先頭に追加）など。AdapterStrategyクラスは、アダプターが入力とどのように統合されるかを決定します。

In [ ]:
# The earlier LinearAdapter has a simple ResidualAddStrategy
# Uncomment below to see the ResidualAddAdapterStrategy definition
# help(linear_adapter.adapter_strategy)

### カスタム Adapter Strategy の作成
残差加算戦略は、単純な操作 $f(x) = x + アダプター(x)$ と考えることができ、ここで $アダプター$ の初期出力は訓練なしで 0 であるべきです。
この処理において、アダプター拡張モデルの出力は本来 $f(x) = x$ であり、したがってモデルは元のモデル（アダプターなし）の性能をそのまま保持する。
-----
以下では、デモンストレーションとして単純な乗算アダプター戦略を作成します。

In [ ]:
from nemo.core.classes.mixins import adapter_mixin_strategies

以下のようにアダプターの特殊メソッド `forward` を実装します：

In [ ]:
# Uncomment to see the definition of the AbstractAdapterStrategy
# help(adapter_mixin_strategies.AbstractAdapterStrategy)

In [ ]:
class MultiplicationAdapterStrategy(adapter_mixin_strategies.AbstractAdapterStrategy):

  def __init__(self, scaling_factor: float = 1.0):
    super().__init__()
    self.scale = scaling_factor

  def forward(self, input: torch.Tensor, adapter: torch.nn.Module, *, module: 'AdapterModuleMixin'):
     # This is the forward method that takes in the previous input (here, its a tensor, but it can be a dictionary, a tuple, a class, anything really).
     # The second argument is the adapter that is currently being applied to this input
     # The final argument is the entire nn.Module that supports adapters.
     # In this case, the final argument would be the entire `MLP` module
     
     # Equivalent to f(x) = x * adapter(x)
     adapter_out = adapter(input)  # compute the adapter output from the input(s)
     result = input * adapter_out

     # Apply scaling factor. Equivalent to f(x) = scale * (x * adapter(x))
     result = self.scale * result
     return result


### Adapter Strategyに対応するdataclassを設計する
このクラスの使用を容易にするため、簡単に戦略オブジェクトを作成できる Dataclass を作成することをお勧めします。以下に使用例を示します：

In [ ]:
from dataclasses import dataclass

@dataclass
class MultiplicationAdapterStrategyConfig:
    scaling_factor: float = 1.0

    # mandatory field
    _target_: str = "{0}.{1}".format(
        MultiplicationAdapterStrategy.__module__, MultiplicationAdapterStrategy.__name__
    )  

### カスタムアダプターコンポーネントの作成
基本的なPyTorchモジュール (`SimpleModule`) とアダプタ戦略 (`MultiplicationAdapterStrategy`) の両方が用意できたので、新しいアダプタコンポーネントを構築できます。
基本的なPyTorchモジュールとアダプターコンポーネントの本質的な違いは`adapter_strategy`にあります。これは、アダプターが元の入力とどのように統合されるかを定義するものです。

In [ ]:
class SimpleModuleAdapter(SimpleModule, adapter_modules.AdapterModuleUtil):

  def __init__(self, size: int, adapter_strategy: MultiplicationAdapterStrategy = None):
    """
    The input arguments should match the original module so you can pass the inputs to the module.
    It should also accept an adapter strategy.

    We will then use the method `setup_adapter_strategy()` to prepare the component to be used as an adapter.
    Note: Passing None to the strategy will let it pick a default strategy provided by the method
    `get_default_strategy_config()`.
    """
    super().__init__(size=size)

    # Prepare the adapter strategy
    self.setup_adapter_strategy(adapter_strategy)

    # Initialize the weights to be 0 at init
    self.reset_parameters()

  # Note: In this case, because we didn't add new modules, nor change how the original forward works
  # We dont need to subclass and override forward() !
  
  def reset_parameters(self):
    # We normally want an adapter at initialization to have no effect on the output
    # Therefore we replace the random uniform with a simple identity matrix, which will cause
    # the output of the adapter to match the input
    with torch.no_grad():
      self.model[0].weight = torch.nn.Parameter(torch.eye(self.size))
  

  def get_default_strategy_config(self) -> 'dataclass':
    """
    Make the default adapter strategy of this component be the `MultiplicationAdapterStrategy()`  
    """
    return MultiplicationAdapterStrategyConfig()

-----アダプターが期待通りに動作するかどうかを素早くテストしてみましょう

In [ ]:
simple_adapter = SimpleModuleAdapter(size=5)
multiplication_strategy = simple_adapter.adapter_strategy
x = torch.randn(1, 5)
adapter_x = simple_adapter(x)
output = multiplication_strategy(input=x, adapter=simple_adapter, module=None)  # Normally you would pass the module here, but in this example can be skipped.
print("Original input :", x)
print("Adapter output :", adapter_x)
print("Strategy output:", output)

元の入力値がアダプターを通過すると、元の値が問題なく返され、その後アダプター戦略によって2つの値が乗算されます（実質的に入力値の二乗を計算することになります）。
これはカスタムアダプターを作成する十分なデモンストレーションであり、通常は要素ごとの乗算をアダプター戦略として実行することはありません。通常は、戦略の出力が初期化時に少なくとも元の初期化値と等しくなることを好みます。

### Adapterコンポーネントに対応するdataclassを設計する
このアダプタコンポーネントの使用をより簡単にするため、コンポーネントを簡単に作成できるデータクラスを用意することをお勧めします。以下に使用例を示します：

In [ ]:
from typing import Optional

@dataclass
class SimpleModuleAdapterConfig:
    size: int
    adapter_strategy: Optional[MultiplicationAdapterStrategyConfig] = None

    # mandatory field
    _target_: str = "{0}.{1}".format(
        SimpleModuleAdapter.__module__, SimpleModuleAdapter.__name__
    )  

## アダプターモジュールの追加
`MLP` は `AdapterModuleMixin` を継承しているため、新しいアダプターを追加するなど、アダプターモジュールを操作するための一連のメソッドも継承しています。
ユーザーがアダプターを追加したい場合、`add_adapter()` 関数を呼び出し、2つの特定の引数 `name` と `cfg` を指定します。
引数 -- `name`: モジュールの場合は**ローカルで一意**、モデルの場合は**グローバルで一意**である文字列名を指定します。また、アダプターが特定のモジュールにのみ属することを指定するために「:」記号を使用することも可能です（この使用方法についてはチュートリアルの終盤で詳しく説明します）。- `cfg`: データクラスまたは OmegaConf 設定オブジェクトで、`_target_` 属性がアダプターモジュールのクラスパスを指し示すほか、必要に応じて追加の属性も含まれます。

In [ ]:
mlp.add_adapter(name='adapter_1', cfg=adapter_modules.LinearAdapterConfig(in_features=dim, dim=5))

In [ ]:
# Now check the new parameter count of this MLP module, it should be higher than the previous count
print("New param count :", mlp.num_weights)

-----
**注意**: 必要な数だけアダプターを追加できます！このチュートリアルでは1つだけ追加しますが、通常は専門化したいタスクごとに1つのアダプターを追加することをお勧めします。
また、複数のアダプターを同時に訓練することは可能ですが（多数のアダプターを追加し、すべて有効化してから凍結解除する方法）、各タスクに対して1つのアダプターのみを訓練することをお勧めします。

-----**注意**: 同じアダプターを複数追加しようとすると、以下のエラーメッセージが表示されます！
注意：アダプター名はモジュールレベルで**ローカルに**一意である必要があり、モデルレベルで**グローバルに**一意である必要があります！

In [ ]:
# Uncomment to see the error message - 
# mlp.add_adapter(name='adapter_1', cfg=adapter_modules.LinearAdapterConfig(in_features=dim, dim=10))

## 有効化されているすべてのアダプタモジュールを取得
次に、`get_enabled_adapters()` を使用して、現在このモジュールで使用可能なすべての有効化済みアダプターの名前リストを返します。

In [ ]:
mlp.get_enabled_adapters()

## アダプターモジュールの状態を設定
上記の方法で有効化されているアダプタ名を取得できますが、アダプタモジュールを有効化または無効化するかどうかを設定するにはどうすればよいでしょうか？
この目的には、`set_enabled_adapter()` メソッドを使用します。いくつかの引数を指定します：- `name`: アダプターの任意の文字列名を指定できます。このアダプターのみを有効にしたり無効にしたりする際に使用します。`name`を指定しない場合、すべてのアダプターモジュールの状態が新しい値に設定されます。- `enabled`: ブール値で、このアダプタを有効にするかどうかを指定します。
-----
アダプターを有効にするということは、単にそのアダプターのフォワードパスを有効にするだけで、それ以上の意味はありません。アダプター自体の重みを凍結／解凍するものではなく、他のアダプターと組み合わせてより複雑な相互作用を可能にするものです。
例えば、モデルにアダプターを追加し、学習させた後にモデルを保存することができます。復元したモデルはさらに別のアダプターを追加することが可能です。2つ目のアダプターを学習させる前に、ユーザーは元のモデルの出力ではなく、最初のアダプターの出力を利用することを選択できます。これを実現するため、両方のアダプターを有効にしつつ、最初のアダプターの重みを凍結し、2つ目のアダプターのみを学習させることができます。

In [ ]:
# Disable all adapters
mlp.set_enabled_adapters(enabled=False)
print("Enabled adapters :", mlp.get_enabled_adapters())

# Enable just one adapter
mlp.set_enabled_adapters(name="adapter_1", enabled=True)
print("Enabled adapters :", mlp.get_enabled_adapters())

## アダプタモジュールの利用可否 / 有効化状態を確認する
上記の2つの方法を拡張する形で、現在のモジュールにアクティブなアダプタモジュールが存在するかどうかを確認することもできます。そのためには、`is_adapter_available()` 関数を使用できます。

In [ ]:
mlp.is_adapter_available()

## Adapter Functionality Methods
上記のいくつかの方法は、アダプターをモジュールに追加および変更するための機能の中核を形成しますが、追加されたアダプターモジュール自体は使用しません！
したがって、以下の機能メソッドはアダプターを適切に活用するために使用され、ユーザーが明示的にオーバーライドする必要はありません（ただし、特定の特殊なケースで必要な場合を除く）。

### `forward_enabled_adapters()`これらのアダプターを使用するには、`forward_adapter_modules()` メソッドを利用します。
有効化されたアダプタを利用するには、`AdapterModuleMixin`を継承したモジュールはまず有効化されたアダプタが存在するかどうかを確認し、その後このメソッドを呼び出して入力データに対するアダプタモジュールを転送する必要があります。

In [ ]:
# Check `forward_enabled_adapters()`
out = mlp.forward_enabled_adapters(input_data)
print(out.shape)

### `forward_single_enabled_adapter_()`アダプターの順方向処理にカスタムロジックを提供するためにサブクラス化可能なメソッドです。例えば、異なる入力セットを持つアダプターを提供したい場合や、順方向処理を実行する前に特定のアダプタータイプをサポートしているかどうかを確認したい場合などに使用します。
アダプターのタイプを確認し、特定のアダプターに入力を転送する前に追加情報を使用することが役立つ場合があります。

In [ ]:
# Check `forward_single_enabled_adapter_()`
adapter_name = mlp.get_enabled_adapters()[0]  # we have enabled just one adapter
adapter_module = mlp.adapter_layer[adapter_name]  # get the adapter module with this name
adapter_strategy = adapter_module.adapter_strategy  # get the adapter strategy for this adapter

out = mlp.forward_single_enabled_adapter_(input_data, adapter_module, adapter_name=adapter_name, adapter_strategy=adapter_strategy)
print(out.shape)

-----アダプターのフォワードパスとアダプター戦略に関する詳細情報については、アダプターのドキュメントセクションを参照してください。

### `unfreeze_enabled_adapters()`アダプターの主な利点の一つは、モデル全体を訓練する必要がないことです。元のモデル/モジュールの残りを凍結したまま、アダプターモジュール自体を訓練することができます。
これは2つのステップで実行できます -- モデルの最上位レベルで model.freeze() を呼び出す- `unfreeze_enabled_adapters()` を呼び出して、有効化されているアダプターモジュールのみを再帰的に凍結解除します。

In [ ]:
# First setup some utility functions (this is part of NeuralModule)
def freeze(m):
    for param in m.parameters():
      param.requires_grad = False
    m.eval()

def unfreeze(m):
    for param in m.parameters():
      param.requires_grad = True
    m.train()

In [ ]:
freeze(mlp)
print("MLP frozen params :", mlp.num_weights)

In [ ]:
# Check `unfreeze_enabled_adapters()` - param count should be lower than the previous total (original + adapter)
mlp.unfreeze_enabled_adapters()
print("MLP unfrozen adapter params :", mlp.num_weights)

# 中級レベルモジュールにおけるアダプターサポート
上記では、`AdapterModuleMixin`を介して単純な`nn.Module`に追加される様々なメソッドと機能について議論しました。ただし、このモジュールはモデルにおける最も基本的な構成要素でした。次に、中間モジュールから下位モジュールへの呼び出しを「ディスパッチ」する方法について見ていきます。
このチュートリアルでは簡潔さを重視し、可能な限り最小限のコード変更に留めます。ただし、下位レベルのモジュールへの中間層ディスパッチをより洗練された形で処理することは十分に可能です。

## 設定ファイル経由でインスタンス化される中間モジュール
現在、私たちは3段階のモデルを採用しています：
`最上位モデル (SimpleModel) → 中間レベルモジュール (ResidualMLP) → 最下位レベルモジュール (MLP)` という構成になっています。
-----
お気づきかもしれませんが、以前のプリマーチュートリアル（NeMo Model Primer）では、モデルが中間モジュールをインスタンス化するために設定を使用することを推奨しています。これにより、ユーザーは設定を介して同等のモジュールを入れ替えることができ、コードの変更を最小限に抑えつつ、モデル自体の機能を最大限に活用できます。
このような「最終版に近い」モジュールについては、元のモジュールを直接修正するのではなく、元のモジュールを拡張する形で別のAdapter対応モジュールを作成することをお勧めします。これは単に元のモジュールコードを乱雑にしないための好みであり、ユーザーが無視しても構いません。
このガイドでは、ベストプラクティスを実践するための推奨設定を紹介します。

## Adapter対応の「Penultimate」モジュールを作成する
まず、新しいAdapter互換モジュールを別のクラスとして作成します。

In [ ]:
# NOTE: We subclass the original ResidualMLP, and add in the AdapterModuleMixin too
class ResidualMLPAdapter(ResidualMLP, adapter_mixins.AdapterModuleMixin):
  pass

## アダプターメソッドのオーバーライド
次に、いくつかのアダプタメソッドをオーバーライドし、これらのメソッドを`ResidualMLP`モジュール内のすべての`MLP`ブロックにディスパッチします。
これにより、`MLP`モジュール内で状態を作成/更新し、アダプターモジュールを転送します！

In [ ]:
from typing import List, Optional

class ResidualMLPAdapter(ResidualMLP, adapter_mixins.AdapterModuleMixin):
  def add_adapter(self, name: str, cfg: DictConfig):
      # call the same method on each `MLP` layer, collecting results
      for layer in self.layers:
        layer.add_adapter(name, cfg)
      
  def get_enabled_adapters(self) -> List[str]:
      # call the same method on each `MLP` layer, collecting results
      enabled_adapters = set([])
      for layer in self.layers:
        names = layer.get_enabled_adapters()
        enabled_adapters.update(names)
      return list(enabled_adapters)
  
  def set_enabled_adapters(self, name: Optional[str], enabled: bool):
      # call the same method on each `MLP` layer, collecting results
      for layer in self.layers:
        layer.set_enabled_adapters(name, enabled)
  
  def is_adapter_available(self) -> bool:
      # call the same method on each `MLP` layer, collecting results
      is_available = any([layer.is_adapter_available() for layer in self.layers])
      return is_available

## 新しいアダプターを登録する
モジュールにアダプター機能を追加するためにサブクラス化する場合、アダプターレジストリにこれらのモジュールを登録することが不可欠です。これにより、後で便利な多くの機能を利用できるようになります。アダプターレジストリは、後でモデル設定をより簡単に更新するために使用できる基本クラスとアダプター互換クラスのグローバルコレクションです。
以下の手順を実行してください：- `get_registered_adapter()` メソッドを使用して、レジストリがベースクラスを持っているかどうかを確認します。- 戻り値が None の場合、`register_adapter()` を使用して基底クラスとその互換性のあるアダプタクラスを登録します。
-----
**注意**: この単純なケースでは、最終モジュールの一つ前のモジュールが実際に中間モジュールとなりますが、現実世界のモデルではさらに多くの中間モジュールが存在する場合があります。このような場合、新しいサブクラスを作成せずに直接 `AdapterModuleMixin` を拡張し、前述の手順に従うことでこれらの中間モジュールを更新できます。このようなケースでは、これらモジュールの登録を省略することも可能です。

In [ ]:
if adapter_mixins.get_registered_adapter(ResidualMLP) is None:
  adapter_mixins.register_adapter(ResidualMLP, ResidualMLPAdapter)

-----
これで中間モジュールのサポートを追加する作業は完了です！すべての中間モジュールに同じ（または類似の）コードを追加するのは一見冗長に思えるかもしれませんが、これは最も単純なディスパッチ方式を実装しているためです。
アダプターを構築する方法にはいくつかの興味深いアプローチが存在します。例えば、注意層専用のアダプター（注意層の前後いずれか、または従来の注意ベースブロックにおける最終フィードフォワード層専用のアダプター）などが挙げられます。このようなアプローチにより、中間層はこれらの機能を下位層に委譲する完全な柔軟性を有しています。

# 最上位モデルにおけるアダプターサポート
最後に、中間モジュールから最下層モジュールへの上記メソッドのディスパッチが完了した後、モデル自体から最初のモジュール（または後方移動時は最後から2番目のモジュール）への最終ディスパッチを実行する必要があります。
このケースでは、これまで使用してきたも `AdapterModuleMixin` とは異なるミックスインクラスを継承します。代わりに、モデルレベルの設定管理機能が組み込まれた `AdapterModelPTMixin` を継承します - これには、アダプター互換モデルの保存と復元を含むモデルレベルの設定管理機能が組み込まれています！


## `AdapterModelPTMixin`の拡張
トップレベルのミックスインを継承する方法は2通りあります：
(1)  現在の Model クラス内で直接拡張する
(2) 追加機能を実装したクラスを作成し、その後そのクラスを継承する。
選択肢 (2) は一見すると目的 (1) を達成するための遠回りな方法のように思えるかもしれません。しかし、これはアダプター管理のロジックを複雑な Model コードベースの外部に保持するためです。Model 自体は、モジュールの設定、データローダー、最適化器/スケジューラ、損失関数、評価指標、そして PyTorch Lightning の「ステップ」（訓練、検証、テストの各ステップ）といった多くの重要な詳細事項に関与しているためです。
-----このチュートリアルでは明確性の観点からオプション (2) を採用します。また、各ステップごとに新しいサブクラスを作成することには透明性を確保する目的があります（同時に大量の情報を提示して読者に負担をかけないためでもあります）。これらの手順はすべて、単一の新しいクラス内で実行することが推奨されます。

In [ ]:
class SimpleModelAdapter(adapter_mixins.AdapterModelPTMixin):
  pass

In [ ]:
help(adapter_mixins.AdapterModelPTMixin)

## 選択的なディスパッチのためのメソッドのオーバーライド
アダプター呼び出しを次のモジュールに振り分ける方法には明らかな違いがありました。これは、モジュールが同質的で典型的な動作を共有していたためです。
しかし、後続のレイヤーがモデルレベルで標準的な動作を共有する理由はありません。`encoder` と `decoder` Transformer レイヤーの観点で考えてみてください - これらは根本的に異なるモジュールです！したがって、それらのアダプターが類似している必要がある理由は何でしょうか？
最上位レベルでは、ユーザー入力を活用して、このような論理的に異種のコンポーネント向けのアダプターをどのように構築するかを決定できます。以下のセクションでは、**グローバル**および**モジュール**レベルのアダプターを活用して、Modelの各**コンポーネント**向けのアダプターの動作と構築を分離する方法について説明します。

## setup_adapters() のオーバーライド
モデルを復元する際には、内部に含まれるすべてのモジュールのパラメータを慎重にロードする必要があります。これまで、torch.nn.Module に Adapter の情報と機能を追加することはできましたが、これらの情報はどこにも保存していませんでした。
したがって、ノートブックを閉じて保存済みチェックポイントを復元しようとすると、復元は失敗します。新しいモデルには以前に追加したアダプターの情報が含まれていないため、state dictのマッチングが失敗します。
この問題は、`setup_adapters()` をオーバーライドし、Model コンストラクタ内でそれを呼び出すことで解決できます。

In [ ]:
# Import the class explicitly to make instance checks easier
from nemo.core.classes.mixins.adapter_mixins import AdapterModuleMixin

class SimpleModelAdapterSetupAdapters(SimpleModelAdapter):
  def setup_adapters(self):
    # First check that any of the modules support adapters or not
    supports_adapters = False

    # Check the inheriting class' modules supports adapters or not
    if hasattr(self, 'encoder') and isinstance(self.encoder, AdapterModuleMixin):
        supports_adapters |= True

    if hasattr(self, 'decoder') and isinstance(self.decoder, AdapterModuleMixin):
        supports_adapters |= True

    # If any class supports it, try to restore adapters
    if supports_adapters:
        super().setup_adapters()

-----
このステップでは、作成したモジュールのいずれかがアダプタをサポートしているかどうかを確認します。もしサポートしているモジュールがあれば、super()メソッドを呼び出して、必要に応じてアダプタを復元しようとします。

## add_adapter() のオーバーライド
次に、`add_adapter`をオーバーライドします。コードに進む前に、まずNeMoでサポートされているアダプターの種類について議論する必要があります。
- `Global Adapters`: これらのアダプターは名称と機能の両面で全てのサポート対象モジュールと共通しています。複数のモデルコンポーネント間で単一のアダプターを共有できる場合に特に有用です。例えば、エンコーダーとデコーダーは同じアダプターを共有しています。- `Module Adapters`: これらのアダプタは特定のモジュール専用であり、そのためモデルの複数コンポーネント間で名前を共有することはできません。アダプタ名は `{module_name}:{adapter_name}` という形式で指定されます。
**注意**: モジュールアダプタを追加した後、そのアダプタは名前の `adapter_name` 部分だけで参照できます。`module_name` を再度指定する必要はありません。なぜなら、すべてのアダプタ名は Model レベルでグローバルに一意であることが保証されているからです。

-----
ユーザーがサポートすべきアダプターは、`Global Adapters`、`Module Adapters`、あるいはその両方です。本チュートリアルでは、両方をサポートするとともに、`encoder`用の`Default Module Adapter`も追加サポートします。
**注意**: `Global` アダプターと `Module` アダプターを区別しやすくするため、便利なメソッド `resolve_adapter_module_name_(name)` の使用を推奨します。使用可能なアダプターモジュールを判定するには、プロパティ `adapter_module_names` の使用をお勧めします。

In [ ]:
class SimpleModelAdapterAddAdapter(SimpleModelAdapterSetupAdapters):

  def add_adapter(self, name: str, cfg: DictConfig):
      # Setup the config first. At the model level, super does not automatically call any of the subclass methods
      # It just sets up the model.cfg for users
      super().add_adapter(name, cfg)

      # Resolve module name and adapter name
      module_name, adapter_name = self.resolve_adapter_module_name_(name)

      # Try to retrieve global adapter config
      global_config = self._get_global_cfg()

      # forward the method call to the individual modules
      # If module name is empty, it is a default and global adapter, otherwise it is a module adapter
      if (module_name == '' and global_config.get('encoder_adapter', True)) or (module_name == 'encoder'):
          self.encoder.add_adapter(name, cfg)

      if (module_name == '' and global_config.get('decoder_adapter', False)) or (module_name == 'decoder'):
          self.decoder.add_adapter(name, cfg)
    
  def resolve_adapter_module_name_(self, name: str) -> (str, str):
      # resolve name and module
      module_name, adapter_name = super().resolve_adapter_module_name_(name)

      # '' as module name means "default module"
      # assert that the module name (if provided) is valid - default, encoder or decoder
      valid_module_names = self.adapter_module_names  # Get the list of supported adapter modules from property
      if module_name not in valid_module_names:
          raise ValueError(f"Provided module name `{module_name}` is not in valid list : {valid_module_names}")

      return (module_name, adapter_name)

  def _get_global_cfg(self):
      # Utility method to get a default "global" adapter config (can be given any value by the user in this config)
      global_config = DictConfig({})
      if 'adapters' in self.cfg and self.adapter_global_cfg_key in self.cfg.adapters:
          global_config = self.adapter_cfg[self.adapter_global_cfg_key]
      return global_config

  @property
  def adapter_module_names(self) -> List[str]:
      module_names = super().adapter_module_names  # "Default" adapter module: ''
      module_names.extend(['encoder', 'decoder'])  # Add support for `encoder` and `decoder` modules
      return module_names


-----
このコード量が多いため、分解して説明します。まず定義されているユーティリティメソッド `_get_global_cfg()` は、モデル設定から「model.cfg.adapters.global_cfg」というサブ設定を取得しようとします。この設定はユーザーが定義するもので、必要に応じて任意のロジックを設定するために使用できます。見つからない場合は、代わりにデフォルトの辞書が作成されます。
次に、`resolve_adapter_module_name_(name)` メソッドをオーバーライドします。このベースクラスのメソッドは文字列名を受け取り、それを `module_name` と `adapter_name` に分割しようとします。このメソッドをオーバーライドして、有効な `module_name` が存在することを断言します。
-----
最後に、`add_adapter` メソッドをオーバーライドします。まず super() を呼び出して設定を更新します。次に、オーバーライドした `resolve_adapter_module_name_(name)` メソッドを呼び出して提供されたアダプタ名が有効かどうかを確認します。その後、モデル設定に存在する場合はアダプタの `global_cfg` を取得します。
この情報をもとに、必要に応じてアダプターを追加できるようになりました。「ユーザー定義」ロジックを実装しており、以下のいずれかの条件が満たされた場合にエンコーダーアダプターを追加します。- ユーザーがデフォルトのモジュール名を持つ「Global」アダプターを指定するか、あるいは`global_cfg.encoder_adapter`の値をTrueに設定している場合（デフォルトではTrue）、これは少なくともエンコーダーアダプターがデフォルトで必ず追加されることを意味します。- ユーザーが `decoder` モジュール名を持つ `Module` アダプターを提供するか、明示的に `global_cfg.decoder_adapter` フラグを True に設定している場合（デフォルトは False）。

## get_enabled_adapters() のオーバーライド
次に、`get_enabled_adapters()`メソッドをオーバーライドします。これは通常、Modelコンポーネントがアダプターをサポートしているかどうかを確認し、サポートしている場合にはそれらのノードの結果を収集して統合するだけで済む簡単な処理です。

In [ ]:
class SimpleModelAdapterGetEnabledAdapters(SimpleModelAdapterAddAdapter):

  def get_enabled_adapters(self) -> List[str]:
      enabled_adapters = super().get_enabled_adapters()

      # Forward the method call to the individual modules
      if isinstance(self.encoder, AdapterModuleMixin):
          encoder_adapters = self.encoder.get_enabled_adapters()
          enabled_adapters.extend(encoder_adapters)

      if isinstance(self.decoder, AdapterModuleMixin):
          decoder_adapters = self.decoder.get_enabled_adapters()
          enabled_adapters.extend(decoder_adapters)

      return enabled_adapters

## set_enabled_adapters() のオーバーライド
上記と同様に、コンポーネントがアダプターをサポートしているかどうかを確認する必要があり、サポートされている場合はそれらのコンポーネントにコールをディスパッチします。
**注意**: ここでは通常の継承チェックではなく、論理チェックを行います。継承チェックだけでは、コンポーネントがアダプターを追加したかどうかまでは判断できません。私たちは「デフォルト/グローバル/モジュールエンコーダー」と「モジュールデコーダー」アダプターの2つの使用ケースがあるため、これらの条件を確認する必要があります。
**注2**: ご存知の通り、set_enabled_adapters() 関数はアダプターの状態をすべて設定する際に `None` を名前として受け取ります。ただし、`resolve_adapter_module_name(name)` メソッドは常に有効な文字列名を受け取る必要があります。そのため、このメソッドに `None` を渡さないように注意してください。

In [ ]:
class SimpleModelAdapterSetEnabledAdapters(SimpleModelAdapterGetEnabledAdapters):

  def set_enabled_adapters(self, name: Optional[str] = None, enabled: bool = True):
      # check if valid model with some adapter support
      super().set_enabled_adapters(name, enabled)

      # Resolve module name and adapter name
      if name is not None:
          module_name, _ = self.resolve_adapter_module_name_(name)
      else:
          module_name = None

      # Try to retrieve global adapter config
      global_config = self._get_global_cfg()

      # Forward the method call to the individual modules
      # Note the OR checks - 
      # if module_name is None - ie explicitly None was passed, set the state for all modules
      # if module name was '' or 'encoder, or if `global_cfg.encoder_adapter` was true, or module_name was '' or 'encoder', forward to encoder.
      # if `global_cfg.decoder_adapter` was true, or module_name was 'decoder', forward to decoder.
      # The user can chose to simplify this logic, or add more complex logic as required.
      if name is None or global_config.get('encoder_adapter', True) or module_name in ('', 'encoder'):
        if self.encoder.is_adapter_available():
          self.encoder.set_enabled_adapters(name, enabled)

      if name is None or global_config.get('decoder_adapter', False) or module_name == 'decoder':
        if self.decoder.is_adapter_available():
          self.decoder.set_enabled_adapters(name, enabled)

## `check_valid_model_with_adapter_support_()` のオーバーライド
上記の実装では、モデルのコンポーネントがアダプターをサポートしているかどうかの暗黙的なチェックを行い、その結果を処理しています。これは許容範囲内ですが、無効なアクションの組み合わせについてより厳密なチェックを行い、意味のある警告やエラーを発生させたい場合があります。
この目的のために、`check_valid_model_with_adapter_support()_`メソッドを提供しています。このメソッドはほぼすべてのアダプター操作の前に呼び出され、いくつかの真理を主張しようとします。ユーザーはここで任意のエラーや警告を発生させることで、無効な操作/設定についてユーザーに通知することができます。

In [ ]:
from nemo.utils import logging, logging_mode

class SimpleModelAdapterFinal(SimpleModelAdapterSetEnabledAdapters):

  def check_valid_model_with_adapter_support_(self):
      global_cfg = DictConfig({})
      if self.adapter_global_cfg_key in self.adapter_cfg:
          global_cfg = self.adapter_cfg[self.adapter_global_cfg_key]

      encoder_adapter = global_cfg.get('encoder_adapter', True)
      decoder_adapter = global_cfg.get('decoder_adapter', False)

      if encoder_adapter and not hasattr(self, 'encoder'):
          logging.warning("Encoder not available", mode=logging_mode.ONCE)
      elif encoder_adapter and not isinstance(self.encoder, AdapterModuleMixin):
          logging.warning("Encoder does not support adapters !", mode=logging_mode.ONCE)

      if decoder_adapter and not hasattr(self, 'decoder'):
          logging.warning("Decoder is not available", mode=logging_mode.ONCE)
      elif decoder_adapter and not isinstance(self.decoder, AdapterModuleMixin):
          logging.warning("Decoder does not support adapters !", mode=logging_mode.ONCE)

## モデルの更新
最上位の Model ミックスインクラスを個別に実装した後、元の Model に簡単に組み込むことができます。チュートリアルの都合上、ここでコードを複製しますが、Model をサブクラス化して `__init__` メソッドをオーバーライドすることで同様の機能を実現することも可能です。

In [ ]:
# Note how we added `SimpleModelAdapterFinal` to the class inheritance scheme.
# The only other change is the addition of `self.setup_adapters()` to the __init__ method.
class SimpleModel(ModelPT, SimpleModelAdapterFinal):
    def __init__(self, cfg, trainer=None):
        super().__init__(cfg, trainer=trainer)

        self.encoder = instantiate(cfg.encoder)  # type: ResidualMLP
        self.decoder = instantiate(cfg.decoder)  # type: ResidualMLP
        self.projection = nn.Linear(self.decoder.dim, cfg.out_features)

        # NOTE: The only important change - calling `setup_adapters()` !
        self.setup_adapters()

    def forward(self, x):
        y = self.encoder(x)
        z = self.decoder(y)
        out = self.projection(z)
        return out

    def list_available_models(cls):
        return []

    def setup_training_data(self, train_data_config):
        pass

    def setup_validation_data(self, val_data_config):
        pass

-----
これで完了です！新しいミックスインをサブクラス化し、`setup_adapters()`を呼び出すだけで、必要な変更は完了です！

In [ ]:
old_config = get_model_config(dim)
model = SimpleModel(old_config)
model.summarize()

-----
では、このモデルに `decoder` Moduleアダプターを追加してみましょう。

In [ ]:
# This cell will error out if uncommented
# model.add_adapter("decoder:adapter_1", cfg=adapter_modules.LinearAdapterConfig(in_features=dim, dim=5))

-----
エラーメッセージ `Encoder does not support adapters !` が表示されます。これは、モデルの元の設定 (`old_config`) には `ResidualMLP` クラスへのクラスパスが含まれていますが、`ResidualMLPAdapter` クラスへのパスは含まれていないためです！
これは簡単に修正可能です。なぜなら、このクラスはすでに適切に登録済みだからです（「新しいアダプターを登録する」サブセクションを参照）

In [ ]:
def get_adapter_model_config() -> DictConfig:
  config = get_model_config()

  # Find the metadata in the registry, and get the correct adapter capable class path
  enc_adapter_metadata = adapter_mixins.get_registered_adapter(config.encoder._target_)
  if enc_adapter_metadata is not None:
      print("Updated encoder to support adapters !")
      config.encoder._target_ = enc_adapter_metadata.adapter_class_path

  # Find the metadata in the registry, and get the correct adapter capable class path
  dec_adapter_metadata = adapter_mixins.get_registered_adapter(config.decoder._target_)
  if dec_adapter_metadata is not None:
      print("Updated decoder to support adapters !")
      config.decoder._target_ = dec_adapter_metadata.adapter_class_path

  return config

In [ ]:
new_config = get_adapter_model_config()
model = SimpleModel(new_config)
model.summarize()

-----では、`decoder` Module Adapterを再度追加してみましょう：

In [ ]:
model.add_adapter('decoder:adapter_1', cfg=adapter_modules.LinearAdapterConfig(in_features=dim, dim=5))

-----複数のログメッセージが表示され、`adapter_1`が追加されたことが示されます（各ブロックごとに1つずつ）。次に、アダプタが正しいモジュールに存在するかどうかを確認します -

In [ ]:
print("Encoder adapter available :", model.encoder.is_adapter_available())
print("Decoder adapter available :", model.decoder.is_adapter_available())
print("Decoder adapter(s) :", model.decoder.get_enabled_adapters())

In [ ]:
model.summarize()

## Adapterのトレーニング準備
最終的に、モデルにアダプター機能が追加されたことで、モデルの他の部分を凍結したままアダプターのみを訓練できるようになりました。
次のセクションでは、このセットアップを実行する方法を説明します。

In [ ]:
# disable all adapters, enable just one adapter that we want to train
model.set_enabled_adapters(enabled=False)
model.set_enabled_adapters('adapter_1', enabled=True)  # note : we directly use the adapter_name of adapter_1

# freeze all the weights, unfreeze just the enabled adapters
model.freeze()
model.unfreeze_enabled_adapters()

print()
model.summarize()

## アダプターの保存と復元
このモデルを実装した今、`model.save_to()` を使用してアダプターサポートを含む完全な NeMo モデルを保存できます。このアダプターモデルを保存してから復元してみましょう。

In [ ]:
model.save_to('full_model.nemo')

In [ ]:
!ls -d -- *.nemo

In [ ]:
new_model = ModelPT.restore_from('full_model.nemo')

In [ ]:
new_model.decoder.get_enabled_adapters()

-----
モデル全体を保存・復元することは可能ですが、必ずしも必要ではありません。ここでアダプターについて考えてみましょう - これらは基本モデルの上に追加されるモジュールです。新しいアダプターを追加するたびに、モデル全体をファイルに保存・復元するのは現実的ではありません（特にモデルのパラメータ数が数十億にもなる場合はなおさらです！）。
次に、`save_adapters()` を使用してモジュール自体のみを個別の .pt ファイルに保存・復元する方法について説明します。

In [ ]:
model.save_adapters('adapters.pt', name=None)

In [ ]:
!du -sh adapters.pt full_model.nemo

-----ご覧の通り、モデル全体はアダプターモジュール自体よりもはるかに大きなサイズになります。これにより、モデル全体のディスク容量を消費することなく、アダプターモジュールのみを他者と共有することが可能になります。
次に、`load_adapters()` を使用してこのようなアダプターチェックポイントを新しいモデルに復元する方法を示します。

In [ ]:
new_config = get_adapter_model_config()
model_2 = SimpleModel(new_config)
model_2.summarize()  # no adapters in basic model with adapter support

In [ ]:
model_2.load_adapters('adapters.pt', name=None, map_location='cpu')

In [ ]:
model_2.freeze()
model_2.unfreeze_enabled_adapters()
model_2.summarize()

In [ ]:
model_2.get_enabled_adapters()

-----上記のモデルに`None`を渡してすべてのアダプターを復元しましたが、チェックポイントから単一のアダプターだけを復元したい場合はどうすればよいでしょうか？その場合は、`load_adapters()`メソッドに`name`引数を渡すことで実現できます。
注意: ここで指定する名前は、アダプター自体を作成する際に使用した名前である必要があります。したがって、モジュールレベルのアダプターの場合は、`module_name` も指定する必要があります。

# 関連文献
アダプターに関する詳細情報については、NeMoドキュメントページのアダプターセクションを参照してください。また、アダプターモジュールを作成する方法について詳しく説明したセクションも含まれています。
アダプターの具体的な使用方法については、以下の関連記事をご参照ください -- [Parameter-Efficient Transfer Learning for NLP](https://arxiv.org/abs/1902.00751)- [Exploiting Adapters for Cross-lingual Low-resource Speech Recognition](https://arxiv.org/abs/2105.11905)- [Efficient Adapter Transfer of Self-Supervised Speech Models for Automatic Speech Recognition](https://arxiv.org/abs/2202.03218)